In [1]:
import re
import sys
import itertools
from pyspark import SparkConf, SparkContext
conf = SparkConf()
sc = SparkContext.getOrCreate(conf=conf)

In [4]:
def makefriendpairs(line):
    split = line.split()
    userid = int(split[0])
    if len(split) == 1:
        friends = []
    else:
        friends = list(map(lambda x: int(x), split[1].split(',')))
    return userid, friends

In [5]:
def makeconnections(friendships):
    userid = friendships[0]
    friends = friendships[1]
    connections = []
    for friend in friends:
        key = (userid, friend)
        if userid > friend:
            key = (friend, userid)
        connections.append((key, 0))
    for pairs in itertools.combinations(friends, 2):
        f1 = pairs[0]
        f2 = pairs[1]
        key = (f1, f2)
        if f1 > f2:
            key = (f2, f1)
        connections.append((key, 1))
    return connections

In [6]:
def makerecommendations(mutual):
    pair = mutual[0]
    count = mutual[1]
    f1 = pair[0]
    f2 = pair[1]
    rec1 = (f1, (f2, count))
    rec2 = (f2, (f1, count))
    return [rec1, rec2]

In [7]:
def sortrecommendations(recs):
    recs.sort(key = lambda x: (-int(x[1]), int(x[0])))
    return list(map(lambda x: x[0], recs))[:10]

In [8]:
users = sc.textFile('soc-LiveJournal1Adj.txt')

pairs = users.map(makefriendpairs)

connections = pairs.flatMap(makeconnections)

mutualconnections = connections.groupByKey().filter(lambda pair: 0 not in pair[1]).map(lambda pair:(pair[0], sum(pair[1])))
mutualconnections.cache()

recommendations = mutualconnections.flatMap(makerecommendations)

sortedrecommendations = recommendations.groupByKey().map(lambda mf: (mf[0], sortrecommendations(list(mf[1]))))

finalanswer = sortedrecommendations.filter(lambda recs: recs[0] in [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]).sortByKey()

finalanswer.saveAsTextFile('Question1')

PythonRDD[25] at RDD at PythonRDD.scala:53


In [ ]:
sc.stop()